<a href="https://colab.research.google.com/github/restrepo/medicion/blob/master/cienciometria/WOS_SCI_SCP.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# WOS+SCI+SCP
Merge the bibliographic datasets for Web of Scienca, Scielo and Scopus of the scientific articles of Universidad de Antioquia

For details see [merge.ipynb in Colaboratory](https://colab.research.google.com/github/restrepo/medicion/blob/master/cienciometria/merge.ipynb)

In [0]:
#!git clone https://github.com/restrepo/WOSplus.git

Cloning into 'WOSplus'...
remote: Enumerating objects: 380, done.
remote: Total 380 (delta 0), reused 0 (delta 0), pack-reused 380
Receiving objects: 100% (380/380), 176.34 KiB | 5.19 MiB/s, done.
Resolving deltas: 100% (236/236), done.


In [0]:
import os
if os.getcwd()=='/content':
    !pip install openpyxl xlrd wosplus  > /dev/null

In [0]:
import wosplus as wp
import pandas as pd

##  Configure public links of  files in Google Drive
* If it is a Google Spreadsheet the corresponding file is downloaded as CSV
* If it is in excel or text file the file is downloaded  directly

To define your  own labeled IDs for public google drive files edit the next cell:

In [4]:
%%writefile drive.cfg
[FILES]Ç
UDEA_WOS.xlsx       = 1px2IcrjCrkyu7t78Q7PAE5nzV_yuPt9t
UDEA_SCI.xlsx       = 1pWMY5P72j0Ca6D-cm7dn7Q4TBGTs4PWV
UDEA_SCP.xlsx       = 1ulCsFHzDiTmuL9TH8F58ulh0u8Z2ylKh
UDEA_WOS_SCI_SCP.xlsx   = 1o9otmklgh-0w18Avv2ZTKOXr3vZbjwvj

Writing drive.cfg


##  Load data bases

In [5]:
UDEA=wp.wosplus('drive.cfg')
tmp=UDEA.load_biblio('UDEA_WOS.xlsx') # creates WOS.WOS
tmp=UDEA.load_biblio('UDEA_SCI.xlsx',prefix='SCI') # creates UDEA.SCI
tmp=UDEA.load_biblio('UDEA_SCP.xlsx',prefix='SCP') # creates UDEA.SCP

In [6]:
print('before merge: {}'.format( UDEA.WOS.shape[0]+UDEA.SCI.shape[0]+UDEA.SCP.shape[0] )  )

before merge: 24676


### WOS+SCI

In [7]:
UDEA.mergeall()

....................................................................................................................................................................................................................................

### WOS+SCI+SCP

In [0]:
UDEA.merge(left='WOS_SCI',right='SCP')

................................................................................................................................................................................................................

In [8]:
print( 'after merge: {}'.format(UDEA.WOS_SCI_SCP.shape[0]))

after merge: 15726


In [0]:
UDEA.WOS_SCI_SCP.to_excel('UDEA_WOS_SCI_SCP.xlsx',index=False)

In [10]:
ls -l UDEA_WOS_SCI_SCP.xlsx

-rw-r--r-- 1 root root 65038948 Mar 21 22:02 UDEA_WOS_SCI_SCP.xlsx


See the resulting Excel file by opening the side bar → Files → REFRESH and right click for Download upon the generated  "'UDEA_WOS_SCI_SCP.xlsx'" fille, or just check the corresponsing DataFrame:

In [0]:
UDEA.WOS_SCI_SCP.sample()

,AB,AF,AR,AU,BA,BE,BF,BN,BP,C1,...,SU,TC,TI,Tipo,U1,U2,UT,VL,WC,Z9
568,,"Vasquez, E.\nFranco, J. L.\nOrrego, J. C.\n",,"Vasquez, E\nFranco, JL\nOrrego, JC\n",,,,,S126\n,"[Vasquez, E.; Franco, J. L.; Orrego, J. C.] Un...",...,3\n,0.0,Subcutaneous (SCIg) Gammaglobulin for the Trea...,WOS,0.0,0.0,WOS:000347388100064\n,33\n,Immunology\n,0.0


In [0]:
UDEA.WOS_SCI_SCP.to_json('UDEA_WOS_SCI_SCP.json.gz',compression='gzip')

## Load merged database

In [0]:
udea=wp.wosplus('drive.cfg')

In [0]:
#udea.load_biblio('UDEA_WOS_SCI_SCP.xlsx')

In [0]:
UDEA=pd.read_excel('UDEA_WOS_SCI_SCP.xlsx')

In [0]:
UDEA=pd.read_json('UDEA_WOS_SCI_SCP.json.gz',compression='gzip')

In [0]:
UDEA_OLD=udea.read_drive_excel('UDEA_WOS_SCI_SCP.xlsx')

In [0]:
UDEA=UDEA.fillna('')

In [0]:
UDEA_OLD=UDEA_OLD.fillna('')

In [26]:
wp.df_split( UDEA, 'DI',Operator='!=',condition='')[0].shape

(8038, 151)

In [29]:
wp.df_split( UDEA_OLD, 'DI',Operator='!=',condition='')[0].shape

(7993, 154)

Checks

In [55]:
UDEA.Tipo.unique()

array(['WOS', 'SCI_SCP', 'WOS_SCP', 'WOS_SCI_SCP', 'SCP', 'WOS_SCI',
       'SCI'], dtype=object)

In [56]:
for t in UDEA.Tipo.unique():
    print( '{}:{}'.format( t, UDEA[ UDEA.Tipo==t].shape[0] ) )

WOS:1884
SCI_SCP:1622
WOS_SCP:5824
WOS_SCI_SCP:773
SCP:2584
WOS_SCI:147
SCI:2892


In [59]:
for t in UDEA.Tipo.unique():
    print( '{}:{}'.format( t, UDEA_OLD[ UDEA_OLD.Tipo==t].shape[0] ) )

WOS:1737
SCI_SCP:1473
WOS_SCP:5971
WOS_SCI_SCP:817
SCP:2362
WOS_SCI:103
SCI:3041


In [0]:
import numpy as np

In [39]:
np.setdiff1d(UDEA[UDEA.DI!=''].DI.values,UDEA_OLD[UDEA_OLD.DI!=''].DI.values)

array(['10.1016/j.rca.2015.03.008', '10.1016/j.rca.2016.02.014',
       '10.1016/j.rca.2016.05.005', '10.1016/j.rccar.2016.09.003',
       '10.1016/j.rcp.2017.06.004', '10.1016/j.uroco.2015.05.009',
       '10.11144/Javeriana.CDR11-73.ards',
       '10.11144/javeriana.CRD11-74.raea',
       '10.12804/revsalud12.2.2014.01',
       '10.15446/agron.colomb.v34n2.54920', '10.15446/dyna.v81n187.40960',
       '10.15446/historelo.v9n17.55495',
       '10.15446/ideasyvalores.v66n164.64873',
       '10.15446/rev.colomb.quim.v45n1.58955', '10.15517/rbt.v65i1.21766',
       '10.17230/co-herencia.12.22.10', '10.17533/udea.iatreia.v28n2a03',
       '10.17533/udea.iatreia.v28n2a04',
       '10.17533/udea.iatreia.v28n2a12.',
       '10.17533/udea.iatreia.v28n3a01', '10.17533/udea.iatreia.v28n3a12',
       '10.17533/udea.iatreia.v28n4a04', '10.17533/udea.iatreia.v28n4a05',
       '10.17533/udea.iatreia.v29n1a0', '10.17533/udea.iatreia.v29n1a01',
       '10.17533/udea.iatreia.v29n1a07', '10.17533/udea.

In [40]:
UDEA_OLD[UDEA_OLD.DI=='10.1016/j.rca.2015.03.008']

,AB,AF,AR,AU,BA,BE,BF,BN,BP,C1,...,SU,TC,TI,Tipo,U1,U2,UT,VL,WC,Z9


In [42]:
UDEA_OLD[UDEA_OLD.SCP_DOI=='10.1016/j.rca.2015.03.008']

,AB,AF,AR,AU,BA,BE,BF,BN,BP,C1,...,SU,TC,TI,Tipo,U1,U2,UT,VL,WC,Z9
11741,,,,"Zamudio-Burbano, Mario Andres\nCasas-Arroyave,...",,,,,,,...,,0,Airway management using ultrasound,SCI_SCP,0,0,,,,0


In [43]:
UDEA[UDEA.DI=='10.1016/j.rca.2015.03.008']

,AB,AF,AR,AU,BA,BE,BF,BN,BP,C1,...,SU,TC,TI,Tipo,U1,U2,UT,VL,WC,Z9
13630,,,,"Zamudio-Burbano M.A., Casas-Arroyave F.D.",,,,,,,...,,0,Airway management using ultrasound [El uso del...,SCP,0,0,,,,0


In [50]:
UDEA_OLD[UDEA_OLD.SCI_TI.str.contains('Airway management')].Tipo

11741    SCI_SCP
Name: Tipo, dtype: object

In [52]:
UDEA[UDEA.SCI_TI.str.contains('Airway management')].Tipo

1566    SCI
Name: Tipo, dtype: object

In [51]:
UDEA[UDEA.SCP_Title.str.contains('Airway management')].Tipo

13630    SCP
Name: Tipo, dtype: object

In [46]:
UDEA[UDEA.SCP_DOI=='10.1016/j.rca.2015.03.008']

,AB,AF,AR,AU,BA,BE,BF,BN,BP,C1,...,SU,TC,TI,Tipo,U1,U2,UT,VL,WC,Z9
13630,,,,"Zamudio-Burbano M.A., Casas-Arroyave F.D.",,,,,,,...,,0,Airway management using ultrasound [El uso del...,SCP,0,0,,,,0


In [47]:
UDEA[UDEA.SCI_DI=='10.1016/j.rca.2015.03.008']

,AB,AF,AR,AU,BA,BE,BF,BN,BP,C1,...,SU,TC,TI,Tipo,U1,U2,UT,VL,WC,Z9


In [19]:
UDEA.drop_duplicates('DI').shape

(8036, 151)

In [0]:
for t in ['WOS','SCI','SCP']:
    print( 'DOIS {}: {}'.format(t,
        wp.df_split( UDEA[ UDEA.Tipo.str.contains(t) ], 'DI',Operator='!=',condition='')[0].shape) )

DOIS WOS: (6025, 154)
DOIS SCI: (1409, 154)
DOIS SCP: (6955, 154)


In [0]:
y.shape

(6025, 154)